### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [1]:
import json
import nltk

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [5]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
       
    ne_index_list_0 = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list_0.append((ne_idx,len(ne)))
    
    if len(main_list) != len(ne_index_list_0):
        continue
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

33752


In [ ]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

In [ ]:
for tk,lab in docs[1]:
    print(tk,lab)

In [6]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [7]:
print(len(data))

33752


In [8]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [9]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X_train))
print(len(X_test))

In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27592
Seconds required: 2.009

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 214297.593862
Feature norm: 1.000000
Error norm: 57013.498881
Active features: 27461
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.716

***** Iteration #2 *****
Loss: 212995.962761
Feature norm: 0.999455
Error norm: 56145.512469
Active features: 25943
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.365

***** Iteration #3 *****
Loss: 172678.017428
Feature norm: 1.710654
Error norm: 109268.534231
Active features: 17455
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #39 *****
Loss: 40204.873664
Feature norm: 27.827901
Error norm: 33621.229472
Active features: 24315
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #40 *****
Loss: 39238.336302
Feature norm: 28.826528
Error norm: 15509.432038
Active features: 24246
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.414

***** Iteration #41 *****
Loss: 38605.886690
Feature norm: 30.064877
Error norm: 25423.934002
Active features: 24156
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.384

***** Iteration #42 *****
Loss: 37889.033799
Feature norm: 31.019546
Error norm: 9028.737605
Active features: 24132
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.376

***** Iteration #43 *****
Loss: 37068.330996
Feature norm: 32.210569
Error norm: 20774.484252
Active features: 24042
Line search trials: 1
Line search step: 1.00

***** Iteration #81 *****
Loss: 23968.166618
Feature norm: 81.406293
Error norm: 3738.158705
Active features: 20246
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.367

***** Iteration #82 *****
Loss: 23879.405410
Feature norm: 83.037001
Error norm: 3879.991123
Active features: 20172
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #83 *****
Loss: 23817.800531
Feature norm: 84.228393
Error norm: 5936.503128
Active features: 20203
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.368

***** Iteration #84 *****
Loss: 23759.475842
Feature norm: 84.941450
Error norm: 2666.215907
Active features: 20243
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.367

***** Iteration #85 *****
Loss: 23696.231933
Feature norm: 86.108000
Error norm: 4424.655074
Active features: 20192
Line search trials: 1
Line search step: 1.000000

***** Iteration #126 *****
Loss: 22900.092908
Feature norm: 104.991413
Error norm: 2613.093759
Active features: 19265
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.381

***** Iteration #127 *****
Loss: 22896.558765
Feature norm: 105.336504
Error norm: 3958.023478
Active features: 19265
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.373

***** Iteration #128 *****
Loss: 22887.526592
Feature norm: 105.546041
Error norm: 2615.831862
Active features: 19242
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.382

***** Iteration #129 *****
Loss: 22884.595444
Feature norm: 105.887868
Error norm: 4013.615047
Active features: 19218
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.387

***** Iteration #130 *****
Loss: 22875.286998
Feature norm: 106.103599
Error norm: 2399.206108
Active features: 19215
Line search trials: 1
Line search step

***** Iteration #165 *****
Loss: 22723.100047
Feature norm: 113.561101
Error norm: 2641.548588
Active features: 19044
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.364

***** Iteration #166 *****
Loss: 22718.901230
Feature norm: 113.677955
Error norm: 1563.357083
Active features: 19027
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #167 *****
Loss: 22716.821409
Feature norm: 113.847878
Error norm: 2549.776111
Active features: 19012
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.366

***** Iteration #168 *****
Loss: 22712.020511
Feature norm: 113.954417
Error norm: 1274.210846
Active features: 19011
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.368

***** Iteration #169 *****
Loss: 22709.875458
Feature norm: 114.117240
Error norm: 2601.102945
Active features: 18991
Line search trials: 1
Line search step

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

因 (O)
服 (O)
药 (O)
困 (O)
难 (O)
， (O)
权 (O)
治 (O)
以 (O)
外 (O)
敷 (O)
法 (O)
， (O)
冀 (O)
杀 (O)
减 (O)
其 (O)
脑 (O)
热 (O)


In [12]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    207603
      B-SYM       0.61      0.46      0.53      3258
      I-SYM       0.52      0.36      0.43      3926
      E-SYM       0.63      0.48      0.54      3233

avg / total       0.95      0.96      0.96    218020

